In [1]:
import os, glob, shutil, argparse, re
import pandas as pd
from pathlib import Path
import numpy as np
import training_info

logdir=training_info.logdir
dir_pattern = logdir+'/{}/{}/{}/{}/epoch{}/b{}/worker{}/thread{}/'
aug_pattern = logdir+'/{}/{}/'
simple_dir_pattern = logdir+'/{}'
logtypes=training_info.logtypes
datasets=training_info.datasets
models=training_info.models
term=training_info.term
output_file="memory.log"

In [2]:
def get_dirname():
    dir_names = {}
    for logtype in logtypes:
        dir_names[logtype]=glob.glob(simple_dir_pattern.format(logtype)+f"/*/*/*/*/*/*/*/")

    return dir_names

def replace_str(target):
    target=target.replace('\n', '')
    target=target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num=replace_str(arr[arr.index(target)+jumpto])
        if num.find("G") != -1:
            num = num.replace("G","")
        elif num.find("M") != -1:
            num = num.replace("M","")
            num = float(num) / 1024
    except:
        print(arr, target, arr[arr.index(target)+jumpto])
        num='NA'
    return num

parsed_dir=training_info.parsed_dir + "/mem"
os.makedirs(parsed_dir, exist_ok=True)

breakdown_col_name=["Used memory (GB)","Free memory (GB)","Shared memory (GB)","Buffer/Page cache (GB)", "Available memory (GB)","Swap space used (GB)","Swap space free (GB)"]
dir_names=get_dirname()
print(dir_names)

{'train_only': ['./log/DDP/train_only/1024batch10/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b256/worker24/thread0/'], 'loadNtrain': ['./log/DDP/loadNtrain/size5/default/resnet18/epoch5/b256/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b256/worker24/thread0/', './log/DDP/loadNtrain/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/'], 'prepNloadNtrain': ['./log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker16/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet1

In [3]:
def parser(dir_names, datasets):
    for logtype in logtypes:
        for logdir in dir_names[logtype]:
            if not os.path.exists(logdir):
                    continue
            logdir_list=logdir.split('/')
            
            dataset=find_value(logdir_list, logtype)
            aug=find_value(logdir_list, dataset)
            model=find_value(logdir_list, aug)
            epoch=find_value(logdir_list, model)
            batchsize=find_value(logdir_list, epoch)
            worker=find_value(logdir_list, batchsize)
                             
            parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}_memory"
            
            logfile=logdir +"/"+ output_file
            breakdown_parsed_log = []

            mem_idx=0
            swap_idx=0

            for line in open(logfile, 'r').readlines():
                if line.startswith(f'Mem:'): # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    used_memory = find_value(info,"Mem:",2)
                    free_memory = find_value(info,"Mem:",3)
                    shared_memory = find_value(info,"Mem:",4)
                    buffer_memory = find_value(info,"Mem:",5)
                    available_memory = find_value(info,"Mem:",6)
                    breakdown_parsed_log.append([used_memory,free_memory,shared_memory,buffer_memory,available_memory])
                    mem_idx+=1
                elif line.startswith(f'Swap:'): # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    used_memory = find_value(info,"Swap:",2)
                    free_memory = find_value(info,"Swap:",3)
                    breakdown_parsed_log[swap_idx].extend([used_memory,free_memory])
                    swap_idx+=1
                else:
                    pass

            breakdown_df = pd.DataFrame(breakdown_parsed_log,
                              columns=breakdown_col_name)
            breakdown_df.dropna().to_csv(parsed_dir+"/"+parse_filename+".csv", sep=',', na_rep='NA')

In [4]:
parser(dir_names, datasets)